# Отчет по лабораторной работе 
## по курсу "Искусственный интеллект"

## Нейросети для распознавания изображений


### Студенты: 

| ФИО       | Роль в проекте                     | Оценка       |
|-----------|------------------------------------|--------------|
| Ефимов А.В. | Подготовил датасет |          |
| Keras | Обучил нейросети |       |
| Jupyter Notebook | Написал отчёт |      |

## Результат проверки

| Преподаватель     | Дата         |  Оценка       |
|-------------------|--------------|---------------|
| Сошников Д.В. |              |               |

> *Комментарии проверяющих (обратите внимание, что более подробные комментарии возможны непосредственно в репозитории по тексту программы)*

## Тема работы

Подготовить набор данных: _Буквы Э, Ю, Я_ - и построить несколько нейросетевых классификаторов 
для распознавания рукописных символов.

## Распределение работы в команде

:)

## Подготовка данных

Перед созданием данным сам процесс можно структуризировать, а именно записывать все символы в 
клетки какой-либо сетки, например:

<img src="img/template/graph-paper.jpg" alt="input grid" width="500"/>

Записывая символы в ячейки, можно отчетливо видеть разбиение. Например, для символа _Я_:

<img src="img/original/scan-ya.jpg" alt="ya scan" width="500"/>

Для наименьшей потери качества (такое, как размытие, свет и т.д.) использован обычный принтерный сканер.
Тогда на выходе получаем (почти) такой же лист, заполненный символами для обучения.

Далее эти символы необходимо вызволить. Это можно сделать с помощью `OpenCV` пройдясь
по картинке простым постоянным шагом, но еще удобнее воспользоваться той решеткой, в которую записаны все 
символы, просто вызволив все ячейки в отдельные картинки.

Это можно сделать по следующему алгоритму:

1. Входную картинку преобразовать в grayscale и в бинарную маску;
1. На бинарной маске найти все контуры и вырезать все, площадь которых
   меньше заданного значения (можно подобрать). В итоге должна остаться
   только сетка, без символов и помех;
1. Вырезанная сетка может быть плохого качества, тогда её нужно
   утолщить и поправить (при пропуске этого шага можно получить мусорные 
   значения);
1. Из маски положительных значений удалить маску решетки. Тогда остается
   только маски ячеек и окружающего контура;
1. Найти на этой маске все контуры и исключить те, которые больше заданного
   значения (также можно подобрать) для исключения окружающего решетку контура;
1. Все полученные контуры будут контурами ячеек. Для каждого из них можно 
   найти ограничивающий треугольник, вырезать его из grayscale картинки,
   уменьшить до $32 \times 32$ и вывести в новую картинку.

Две заметки:

1. Вырезаны могут быть картинки не только из grayscale, но и из оригинальной 
   фотографии, но так как интересует только форма, достаточно сохранить только
   монотонную версию;
1. Необходимо проверять, что вписанные символы не задевают решетку, иначе они
   станут частью её контура и, соответственно, будут вырезаны;
1. Итоговую картинку следует инвертировать: в оригинале символ имеет темные,
   отрицательные значения, а границы положительные. Это сильно влияет на точность,
   так как исключается сама форма и сохраняются границы, а нужно наоборот.
   
После всех преобразований итоговые картинки будут иметь вид:

<img src="img/processed/scan-ya-processed.png" alt="Processed ya scan" width="500"/>

Или в сжатом, единичном формате:

![Small sample](img/splits/ya/split1.png)

Эти действия повторяются для каждого из интересующих листов.

Стоит заметить, что такой алгоритм __не__ исключает пустые ячейки, и для этого нужно
либо его расширять, либо удалять их вручную.

## Загрузка данных

Перед вступлением в программную часть сначала следует загрузить все используемые библиотеки:

In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras import layers

И некоторые гиперпараметры: 

In [2]:
img_size = (32, 32)
batch_size = 64
num_epochs = 20
hidden_size = 512

Директория с картинками имеет структуру:

* splits
  * ya
    * split1.png
    * split2.png
    * ...
  * ye
    * split1.png
    * split2.png
    * ...
  * yu
    * split1.png
    * split2.png
    * ...

Все картинки можно загрузить из директории за два прохода: для обучающей и тестирующей выборок - 
с помощью фукнции `image_dataset_from_directory`, которая каждую поддиректорию считает за отдельный класс.

In [3]:
seed = 101 # Needed for validation split
validation_perc = 0.2
data_folder = "img/splits"

train_ds = keras.preprocessing.image_dataset_from_directory(
    data_folder, 
    batch_size=batch_size,
    color_mode="grayscale",
    image_size=img_size,
    seed=seed,
    validation_split=validation_perc, 
    subset='training')
valid_ds = keras.preprocessing.image_dataset_from_directory(
    data_folder, 
    batch_size=batch_size,
    color_mode="grayscale",
    image_size=img_size,
    seed=seed,
    validation_split=validation_perc, 
    subset='validation')

data_example = next(iter(train_ds))
print("Feats:", data_example[0].shape) 
print("Labels:", data_example[1].shape)
print(data_example[1])

Found 1596 files belonging to 3 classes.
Using 1277 files for training.
Found 1596 files belonging to 3 classes.
Using 319 files for validation.
Feats: (64, 32, 32, 1)
Labels: (64,)
tf.Tensor(
[2 2 0 2 1 1 0 0 2 1 2 1 2 2 1 1 0 2 2 1 2 2 0 0 2 0 1 0 2 0 0 1 2 1 1 2 0
 0 0 0 2 1 1 0 2 1 2 2 1 0 0 1 0 1 2 2 0 0 2 0 2 2 0 0], shape=(64,), dtype=int32)


Стандартно, эта функция сразу перемешивает входные данные.

## Обучение нейросети
### Полносвязная однослойная сеть

In [4]:
model = Sequential()
model.add(layers.Flatten())
model.add(layers.Normalization())
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_ds, validation_data=valid_ds, epochs=num_epochs)

Epoch 1/20
20/20 [==============================] - 15s 17ms/step - loss: 20.6052 - accuracy: 0.4377 - val_loss: 3.9136 - val_accuracy: 0.7712
Epoch 2/20
20/20 [==============================] - 0s 3ms/step - loss: 2.2827 - accuracy: 0.8525 - val_loss: 1.9789 - val_accuracy: 0.9122
Epoch 3/20
20/20 [==============================] - 0s 3ms/step - loss: 1.7014 - accuracy: 0.9167 - val_loss: 1.5613 - val_accuracy: 0.9154
Epoch 4/20
20/20 [==============================] - 0s 3ms/step - loss: 0.8566 - accuracy: 0.9508 - val_loss: 1.2283 - val_accuracy: 0.9310
Epoch 5/20
20/20 [==============================] - 0s 3ms/step - loss: 0.7914 - accuracy: 0.9497 - val_loss: 1.0485 - val_accuracy: 0.9342
Epoch 6/20
20/20 [==============================] - 0s 3ms/step - loss: 0.5587 - accuracy: 0.9589 - val_loss: 0.8190 - val_accuracy: 0.9342
Epoch 7/20
20/20 [==============================] - 0s 3ms/step - loss: 0.2993 - accuracy: 0.9729 - val_loss: 0.7704 - val_accuracy: 0.9310
Epoch 8/20
20/20 

Так как входные данные достаточно одинаковые, то уже на однослойной сети достигается 
высокая точность в $96.87\%$.

### Полносвязная многослойная сеть

Но на этом можно не останавливаться и построить сеть, достигающую высокую точность 
за меньшее количество итераций:

In [5]:
model = Sequential()
model.add(layers.Flatten(input_shape=(*img_size, 1)))
model.add(layers.Normalization())
model.add(layers.Dense(hidden_size, activation='relu'))
model.add(layers.Dense(hidden_size, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
normalization_1 (Normalizati (None, 1024)              2049      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 1539      
Total params: 791,044
Trainable params: 788,995
Non-trainable params: 2,049
_________________________________________________________________


Эта сеть состоит из 5 слоев:
* Первые два слоя нужные для сжатия входных данных и их нормализации из
  $[0, 255]$ в $[-1, 1]$. Их параметры необучаемые (зависят только от входных данных)
  или отсутствуют вовсе.
* Следующие два слоя содержут простые линейные функции с активационными функциями `ReLU`.
  Матрица весов первого имеет $(1024 + 1) * 512 = 524800$, второго $(512 + 1) * 512 = 262656$
  (один прибавляется для учитывания bias).
* Последний также простой полносвязный, но с активационной функций `Softmax`, отображающей
  значения нейросети в классы. Матрица весов - $(512 + 1) * 3 = 1539$

За функцию потерь взята `CrossEntropy` функция, работающая на int категориях.
Оптимизатор Адама.

In [6]:
model.fit(train_ds, validation_data=valid_ds, epochs=num_epochs)

Epoch 1/20
20/20 [==============================] - 1s 15ms/step - loss: 16.6523 - accuracy: 0.5958 - val_loss: 1.1436 - val_accuracy: 0.9687
Epoch 2/20
20/20 [==============================] - 0s 7ms/step - loss: 0.6996 - accuracy: 0.9608 - val_loss: 0.2469 - val_accuracy: 0.9812
Epoch 3/20
20/20 [==============================] - 0s 6ms/step - loss: 0.1572 - accuracy: 0.9873 - val_loss: 0.2915 - val_accuracy: 0.9812
Epoch 4/20
20/20 [==============================] - 0s 7ms/step - loss: 0.0485 - accuracy: 0.9909 - val_loss: 0.0967 - val_accuracy: 0.9906
Epoch 5/20
20/20 [==============================] - 0s 7ms/step - loss: 3.7983e-04 - accuracy: 1.0000 - val_loss: 0.0962 - val_accuracy: 0.9843
Epoch 6/20
20/20 [==============================] - 0s 6ms/step - loss: 4.2361e-05 - accuracy: 1.0000 - val_loss: 0.0969 - val_accuracy: 0.9843
Epoch 7/20
20/20 [==============================] - 0s 7ms/step - loss: 5.9329e-06 - accuracy: 1.0000 - val_loss: 0.0960 - val_accuracy: 0.9843
Epoch 

Точность многослойной сети стала равна однослойной уже на первой эпохе и продолжала расти.

### Свёрточная сеть

Но зачем останавливаться на многосвязанных сетях (хотя стоит), если можно ускорить обучение 
за счет учитывания особенностей самих букв.

In [7]:
cnn_model = Sequential()
cnn_model.add(layers.Normalization(input_shape=(*img_size, 1)))
cnn_model.add(layers.Conv2D(8, 5, padding='valid', activation='relu'))
cnn_model.add(layers.MaxPooling2D())
cnn_model.add(layers.Conv2D(16, 5, padding='valid', activation='relu'))
cnn_model.add(layers.MaxPooling2D())
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(128, activation='relu'))
cnn_model.add(layers.Dense(3, activation='softmax'))

cnn_model.compile(optimizer='Adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_2 (Normalizati (None, 32, 32, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 8)         208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        3216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

Слои:
* Первый слой - нормализующий (не тренируется)
* Второй слой - ищет особенности / строит карту особенностей для входных значений
* Третий слой - бъет входящие карты на квадраты (2,2) и возвращает карту из максимумов
  этих квадратов (нужен для сокращения обучающих параметров и упрощения модели)
  (не тренируется)
* Четвертый и пятый слои - тоже самое, что и второй и третий, но уже на выведенных
  ими картах особенностей

Остальные слои схожи с многосвязанной сетью, которой на вход поступают не картинки, 
а карты обнаруженных в них особенностей.

In [8]:
cnn_model.fit(train_ds, validation_data=valid_ds, epochs=num_epochs)

Epoch 1/20
20/20 [==============================] - 2s 23ms/step - loss: 2.4652 - accuracy: 0.7181 - val_loss: 0.1885 - val_accuracy: 0.9781
Epoch 2/20
20/20 [==============================] - 0s 12ms/step - loss: 0.1711 - accuracy: 0.9684 - val_loss: 0.0570 - val_accuracy: 0.9875
Epoch 3/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0236 - accuracy: 0.9915 - val_loss: 0.0166 - val_accuracy: 0.9969
Epoch 4/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0048 - accuracy: 0.9993 - val_loss: 0.0201 - val_accuracy: 0.9969
Epoch 5/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0020 - accuracy: 0.9999 - val_loss: 0.0221 - val_accuracy: 0.9969
Epoch 6/20
20/20 [==============================] - 0s 13ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0256 - val_accuracy: 0.9969
Epoch 7/20
20/20 [==============================] - 0s 13ms/step - loss: 8.8125e-04 - accuracy: 1.0000 - val_loss: 0.0197 - val_accuracy: 0.9969
Epoch 8/2

Предельно максимальная точность достигнута за три эпохи, что значительно лучше первых двух моделей,
причем скорость практически не падает, возможно из-за маленьких размеров датасета.

## Вывод

Так как работа с данными почти ни чем не отличается от предыдущих лабораторных работ, 
то выводы по созданию собственного датасета:
* `OpenCV` - мощная библиотека, позволяющая настроить конвейер от скана листа бумаги с 
  решеткой до множества картинок с содержимым её ячеек (автоматический фотошоп);
* `Keras` имеет метод быстрой загрузки и классификации картинок при условии, что они
  корректно разделены по папкам;
* Однослойная сеть имеет подозрительно высокую точность. Возможно датасет, написанный
  одним человеком, распознается достаточно легко даже если почерк пытаться менять.